# 特征选择

数据预处理后，我们生成了大量的新变量（比如独热编码生成了大量仅包含0或1的变量）。但实际上，部分新生成的变量可能是多余：一方面它们本身不一定包含有用的信息，故无法提高模型性能；另一方面过这些多余变量在构建模型时会消耗大量内存和计算能力。因此，我们应该进行特征选择并选择特征子集进行建模。

## 1. Filter Methods 过滤法

过滤法通过使用一些统计量或假设检验结果为每个变量打分。得分较高的功能往往更加重要，因此应被包含在子集中。以下为一个简单的基于过滤法的机器学习工作流（以最简单的训练-验证-测试这种数据集划分方法为例）。

### Univariate Filter Methods 单变量特征过滤

单变量过滤方法依据单变量统计量或统计检验选择最佳特征。其仅仅考虑单个变量与目标变量的关系（方差选择法仅基于单个变量）。

### 1.1 Variance Threshold 方差选择法

方差选择法删除变量方差低于某个阈值的所有特征。例如，我们应删除方差为零的特征（所有观测点中具有相同值的特征），因为该特征无法解释目标变量的任何变化。

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

In [2]:
# 合成一些数据集用于演示
train_set = np.array([[1,2,3],[1,4,7],[1,4,9]]) # 可见第一个变量方差为0
train_set

array([[1, 2, 3],
       [1, 4, 7],
       [1, 4, 9]])

In [3]:
test_set = np.array([[3,2,3],[1,2,7]]) # 故意将第二个变量方差设为0
test_set

array([[3, 2, 3],
       [1, 2, 7]])

In [4]:
selector = VarianceThreshold()
selector.fit(train_set) # 在训练集上训练
transformed_train = selector.transform(train_set) # 转换训练集

In [5]:
transformed_test = selector.transform(test_set) # 转换测试集

### 1.2 相关系数法

#### Pearson Correlation (regression problem)  皮尔森相关系数 (回归问题)

皮尔森相关系数一般用于衡量两个**连续**变量之间的线性相关性，也可以用于衡量二元变量与目标变量的相关性。故可以将类别变量利用独热编码转换为多个二元变量之后利用皮尔森相关系数进行筛选。

公式：
$r = \frac{\sum_{i=1}^{n}(X_i-\bar{X})(Y_i-\bar{Y})}{\sqrt{(X_i-\bar{X})^2}\sqrt{(Y_i-\bar{Y})^2}}$

In [ ]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest

# 直接载入数据集
from sklearn.datasets import fetch_california_housing
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target # 利用 california_housing 数据集来演示
# 此数据集中，X，y均为连续变量，故此满足使用皮尔森相关系数的条件

# 选择前15000个观测点作为训练集
# 剩下的作为测试集
train_set = X[0:15000,:]
test_set = X[15000:,]
train_y = y[0:15000]

# sklearn 中没有直接的函数可以使用
# 此处将用 scipy.stats.pearsonr函数来实现基于皮尔森相关系数的特征过滤
# 注意 scipy.stats.pearsonr 计算的是两个变量之间的相关系数
# 因sklearn SelectKBest需要，我们将基于scipy.stats.pearsonr 重写允许多特征同时输入的函数 udf_pearsonr

def udf_pearsonr(X, y):
    # 将会分别计算每一个变量与目标变量的关系
    result = np.array([pearsonr(x, y) for x in X.T]) # 包含(皮尔森相关系数, p值) 的列表
    return np.absolute(result[:,0]), result[:,1] 

# SelectKBest 将会基于一个判别函数自动选择得分高的变量
# 这里的判别函数为皮尔森相关系数
selector = SelectKBest(udf_pearsonr, k=2) # k => 我们想要选择的变量数
selector.fit(train_set, train_y) # 在训练集上训练
transformed_train = selector.transform(train_set) # 转换训练集
transformed_train.shape #(15000, 2), 其选择了第一个及第七个变量 

In [6]:
assert np.array_equal(transformed_train, train_set[:,[0,6]]) 

transformed_test = selector.transform(test_set) # 转换测试集
assert np.array_equal(transformed_test, test_set[:,[0,6]]);
# 可见对于测试集，其依然选择了第一个及第七个变量 

In [7]:
# 验算一下我们的结果
for idx in range(train_set.shape[1]):
    pea_score, p_value = pearsonr(train_set[:,idx], train_y)
    print(f"第{idx + 1}个变量和目标的皮尔森相关系数的绝对值为{round(np.abs(pea_score),2)}, p-值为{round(p_value,3)}")
# 应选择第一个及第七个变量 

第1个变量和目标的皮尔森相关系数的绝对值为0.7, p-值为0.0
第2个变量和目标的皮尔森相关系数的绝对值为0.07, p-值为0.0
第3个变量和目标的皮尔森相关系数的绝对值为0.14, p-值为0.0
第4个变量和目标的皮尔森相关系数的绝对值为0.04, p-值为0.0
第5个变量和目标的皮尔森相关系数的绝对值为0.02, p-值为0.011
第6个变量和目标的皮尔森相关系数的绝对值为0.05, p-值为0.0
第7个变量和目标的皮尔森相关系数的绝对值为0.23, p-值为0.0
第8个变量和目标的皮尔森相关系数的绝对值为0.08, p-值为0.0


#### F-Score (regression problem) F-统计量 (回归问题)

F统计量（F-Score）用于检验线性回归模型的整体显著性。在sklearn中，其将对每一个变量分别建立一个一元的线性回归模型，然后分别报告每一个对应模型的F统计量。F-统计量的零假设是该线性模型系数不显著，在一元模型中，该统计量能够反映各变量与目标变量之间的线性关系。因此，我们应该选择具有较高F统计量的特征（更有可能拒绝原假设）。

公式：  
  
$F = \frac{(SST - SSR)/(p - 1)}{SSR/(n - p)} =  \frac{SST - SSR}{SSR/(n - 2)} =  \frac{R^2}{(1 - R^2)(n - 2)} = \frac{\rho ^2}{(1 - \rho ^2)(n - 2)}$  
 
其中:  

$SST = \sum_{i=1}^{n}(y_i - \overline{y}) ^2$  
  
$\overline{y} = \frac{1}{n} \sum_{i=1}^{n}y_i$  
  
$SSR = \sum_{i=1}^{n}(\widehat{y}_i - \overline{y})^2$  
  
$\widehat{y}_i$ 为模型预测值
  
  
SST为总平方和，SSR为回归平方和，p为线性回归自变量数（包括常数项，故在上述的一元线性模型中，p=2），$\rho$为自变量与因变量的线性相关系数，n为总观测数。因上述线性模型为一元线性模型，故可证$\rho^2 = R^2$。 

In [8]:
import numpy as np
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest

# 直接载入数据集
from sklearn.datasets import fetch_california_housing
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target # 利用 california_housing 数据集来演示
# 此数据集中，X，y均为连续变量，故此满足使用F统计量的条件

# 选择前15000个观测点作为训练集
# 剩下的作为测试集
train_set = X[0:15000,:]
test_set = X[15000:,]
train_y = y[0:15000]

# sklearn 中直接提供了函数用于计算F统计量
# SelectKBest 将会基于一个判别函数自动选择得分高的变量
# 这里的判别函数为F统计量
selector = SelectKBest(f_regression, k=2) # k => 我们想要选择的变量数
selector.fit(train_set, train_y) # 在训练集上训练
transformed_train = selector.transform(train_set) # 转换训练集
transformed_train.shape #(15000, 2), 其选择了第一个及第七个变量 
assert np.array_equal(transformed_train, train_set[:,[0,6]]) 

transformed_test = selector.transform(test_set) # 转换测试集
assert np.array_equal(transformed_test, test_set[:,[0,6]]);
# 可见对于测试集，其依然选择了第一个及第七个变量 

In [9]:
# 验算一下我们的结果
for idx in range(train_set.shape[1]):
    score, p_value = f_regression(train_set[:,idx].reshape(-1,1), train_y)
    print(f"第{idx + 1}个变量的F统计量为{round(score[0],2)}, p-值为{round(p_value[0],3)}")
# 故应选择第一个及第七个变量 

第1个变量的F统计量为14111.79, p-值为0.0
第2个变量的F统计量为71.99, p-值为0.0
第3个变量的F统计量为317.04, p-值为0.0
第4个变量的F统计量为23.93, p-值为0.0
第5个变量的F统计量为6.54, p-值为0.011
第6个变量的F统计量为35.93, p-值为0.0
第7个变量的F统计量为846.61, p-值为0.0
第8个变量的F统计量为98.06, p-值为0.0


### 1.3 Mutual Information (regression problem) 互信息 (回归问题)

互信息（Mutual Information）衡量变量间的相互依赖性。其本质为熵差，即$H(X) - H(X|Y)，即知道另一个变量信息后混乱的降低程度$。当且仅当两个随机变量独立时MI等于零。MI值越高，两变量之间的相关性则越强。与Pearson相关和F统计量相比，它还捕获了非线性关系。

公式:  
  
- 若两个变量均为离散变量:  
  
    $I(x, y) = H(Y) - H(Y|X) = \sum_{x\in \mathit{X}}  \sum_{x\in \mathit{Y}} \textit{p}_{(X,Y)}(x,y) \textrm{log}(\frac{\textit{p}_{(X,Y)}(x,y)}{\textit{p}_{X}(x)\textit{p}_{Y}(y)})$  

    $\textit{p}_{(X,Y)}(x,y)$ 为x和y的联合概率质量函数 (PMF)， $\textit{p}_{X}(x)$则为x的联合概率质量函数 (PMF)。  
  
- 若两个变量均为连续变量:  

    $I(X, Y) = H(Y) - H(Y|X) = \int_X \int_Y  \textit{p}_{(X,Y)}(x,y) \textrm{log}(\frac{\textit{p}_{(X,Y)}(x,y)}{\textit{p}_{X}(x)\textit{p}_{Y}(y)}) \, \, dx dy$  
    
    $\textit{p}_{(X,Y)}(x,y)$ 为x和y的联合概率密度函数 (PDF)，$\textit{p}_{X}(x)$则为x的概率密度函数 (PDF)。连续变量情形下，在实际操作中，往往先对数据离散化分桶，然后逐个桶进行计算。
  

但是实际上，一种极有可能的情况是，x和y中的一个可能是离散变量，而另一个是连续变量。因此在sklearn中，它基于[1]和[2]中提出的基于k最临近算法的熵估计非参数方法。   
   
[1] A. Kraskov, H. Stogbauer and P. Grassberger, “Estimating mutual information”. Phys. Rev. E 69, 2004.  
[2] B. C. Ross “Mutual Information between Discrete and Continuous Data Sets”. PLoS ONE 9(2), 2014. 

In [10]:
import numpy as np
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest

# 直接载入数据集
from sklearn.datasets import fetch_california_housing
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target # 利用 california_housing 数据集来演示
# 此数据集中，X，y均为连续变量，故此满足使用MI的条件

# 选择前15000个观测点作为训练集
# 剩下的作为测试集
train_set = X[0:15000,:].astype(float)
test_set = X[15000:,].astype(float)
train_y = y[0:15000].astype(float)

# KNN中的临近数是一个非常重要的参数
# 故我们重写了一个新的MI计算函数更好的来控制这一参数
def udf_MI(X, y):
    result = mutual_info_regression(X, y, n_neighbors = 5) # 用户可以输入想要的临近数
    return result

# SelectKBest 将会基于一个判别函数自动选择得分高的变量
# 这里的判别函数为F统计量
selector = SelectKBest(udf_MI, k=2) # k => 我们想要选择的变量数
selector.fit(train_set, train_y) # 在训练集上训练
transformed_train = selector.transform(train_set) # 转换训练集
transformed_train.shape #(15000, 2), 其选择了第一个及第八个变量
assert np.array_equal(transformed_train, train_set[:,[0,7]])

transformed_test = selector.transform(test_set) # 转换测试集
assert np.array_equal(transformed_test, test_set[:,[0,7]]);
# 可见对于测试集，其依然选择了第一个及第八个变量

In [11]:
# 验算上述结果
for idx in range(train_set.shape[1]):
    score = mutual_info_regression(train_set[:,idx].reshape(-1,1), train_y, n_neighbors = 5)
    print(f"第{idx + 1}个变量与因变量的互信息为{round(score[0],2)}")
# 故应选择第一个及第八个变量

第1个变量与因变量的互信息为0.38
第2个变量与因变量的互信息为0.03
第3个变量与因变量的互信息为0.1
第4个变量与因变量的互信息为0.03
第5个变量与因变量的互信息为0.02
第6个变量与因变量的互信息为0.09
第7个变量与因变量的互信息为0.37
第8个变量与因变量的互信息为0.46


#### F-Score (classification problem) F-统计量 (分类问题)

在分类机器学习问题中，若变量特征为类别特征，则我们可以使用独热编码配合上述chi2方法选择最重要的特征。但若特征为连续变量，则我们可以使用ANOVA-F值。ANOVA F统计量的零假设是若按目标变量（类别）分组，则连续变量的总体均值是相同的。故我们应选择具有高ANOVA-F统计量的连续变量，因为这些连续变量与目标变量的关联性强。

公式：  
  
$F = \frac{MSB}{MSE} = \frac{ \frac{SS(between)}{m-1}}{ \frac{SS(error)}{n-m}}$   
  
其中，SS(between)为组间的平方和，即组均值和总体均值之间的平方和。 SS(error)是组内的平方和，即数据与组均值之间的平方和。 m是目标变量的总类别数，n是观测数。

In [20]:
import numpy as np
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

# 直接载入数据集
from sklearn.datasets import load_iris # 利用iris数据作为演示数据集
iris = load_iris()
X, y = iris.data, iris.target
# 此数据集中，X为连续变量，y为类别变量
# 满足ANOVA-F的使用条件

# iris 数据集使用前需要被打乱顺序
np.random.seed(1234)
idx = np.random.permutation(len(X))
X = X[idx]
y = y[idx]

# 选择前100个观测点作为训练集
# 剩下的作为测试集
train_set = X[0:100,:]
test_set = X[100:,]
train_y = y[0:100]

# sklearn 中直接提供了函数用于计算ANOVA-F
# SelectKBest 将会基于一个判别函数自动选择得分高的变量
# 这里的判别函数为F统计量
selector = SelectKBest(f_classif, k=2) # k => 我们想要选择的变量数
selector.fit(train_set, train_y) # 在训练集上训练
transformed_train = selector.transform(train_set) # 转换训练集
transformed_train.shape #(100, 2), 其选择了第三个及第四个变量 
assert np.array_equal(transformed_train, train_set[:,[2,3]]) 

transformed_test = selector.transform(test_set) # 转换测试集
assert np.array_equal(transformed_test, test_set[:,[2,3]]);
# 可见对于测试集，其依然选择了第三个及第四个变量 

In [21]:
# 验证上述结果
for idx in range(train_set.shape[1]):
    score, p_value = f_classif(train_set[:,idx].reshape(-1,1), train_y)
    print(f"第{idx + 1}个变量与因变量的ANOVA-F统计量为{round(score[0],2)}，p值为{round(p_value[0],3)}")
# 故应选择第三个及第四个变量 

第1个变量与因变量的ANOVA-F统计量为91.39，p值为0.0
第2个变量与因变量的ANOVA-F统计量为33.18，p值为0.0
第3个变量与因变量的ANOVA-F统计量为733.94，p值为0.0
第4个变量与因变量的ANOVA-F统计量为608.95，p值为0.0


### Embedded Methods 嵌入方法

过滤法的特征选择过程与后续的机器学习模型无关，因此过滤法可能导致较差的模型性能。

封装方法利用预定义的有监督的机器学习模型来选择最佳功能。但是，由于它们需要在大量可能的特征子集上多次训练模型，因此尽管它们通常会导致更好的性能，但它们同时也需要较长的处理时间。

嵌入式方法将特征选择过程嵌入到机器学习模型中，即利用机器学习来为每一个特征打分。嵌入式方法在创建模型时即完成了对特征子集的选择。因此，与过滤法相比，它们往往具有更好的性能。与封装方法相比，它们节省了大量的处理时间和计算能力。

**三种方法的一个简单对比**.  
   
|方面 | 过滤法 | 封装法	| 嵌入法
|--|--|--|--|
|是否需模型参与| 否 | 是 | 是 |
|是否需要交叉验证 |	可能(可利用交叉验证选择保留的特征数目) | 是 | 可能(可利用交叉验证选择保留的特征数目)
|处理时间 |	短 | 长 | 中等
|对参与模型的限制|	无 | 无 | 是 (嵌入法使用的模型为线性模型或树类模型)

![image](../images/嵌入法工作流.png)

### 基于正则化模型的方法

许多机器学习模型在其损失函数中引入了正则项（L1正则或L2正则），以防止过拟合问题。线性模型（例如线性向量支持机，逻辑回归，线性回归）中的L1正则项能够有效地将某些特征的特征系数缩小为零，从而实现解的稀疏。因此，基于带正则项线性模型的特征系数，我们可以为特征打分。系数越高，往往该特征在线性模型中越重要。

我们可以使用sklearn SelectFromModel函数删除特征系数低或为零的特征。

#### Lasso Regression (Linear Regression with L1 Norm) 套索回归

In [23]:
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso # 我们也可以使用带L2正则项的岭回归

# 直接载入数据集
from sklearn.datasets import fetch_california_housing
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target # 利用 california_housing 数据集来演示

# 选择前15000个观测点作为训练集
# 剩下的作为测试集
train_set = X[0:15000,:]
test_set = X[15000:,]
train_y = y[0:15000]

clf = Lasso(normalize=True, alpha = 0.001)  
# 在进行线性回归前，我们需要先对变量进行缩放操作，否则回归系数大小无法比较
# alpha控制正则效果的大小，alpha越大，正则效果越强

clf.fit(train_set, train_y) # 在训练集上训练
np.round(clf.coef_ ,3) 

array([ 0.346,  0.003, -0.   , -0.   , -0.   , -0.   , -0.033,  0.   ])

In [24]:
selector = SelectFromModel(clf, prefit=True, threshold=1e-5)
# 阈值被设置为1e-5，因此绝对系数低于1e-5的特征将被删除
# 我们还可以设置max_features参数以选择最重要的前几个特征

transformed_train = selector.transform(train_set) # 转换训练集
transformed_test = selector.transform(test_set) #转换测试集

assert np.array_equal(transformed_train, train_set[:,[0,1,6]]) 
# 选择第一个，第二个及第七个变量
assert np.array_equal(transformed_test, test_set[:,[0,1,6]]) 